# Homewor 4 - MLE, Gaussian Mixture, and SVM

## CSCI 4622 - Fall 2022

***
**Name**: $<$insert name here$>$
***

This assignment is due on Canvas by **11.59 PM on Friday, October 28th**.
Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc.
Your solutions to analysis questions should be done in Markdown directly below the associated question.
Remember that you are encouraged to discuss the problems with your classmates and instructors,
but **you must write all code and solutions on your own**, and list any people or sources consulted.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import helpers
import data

### [20 points] Problem 1 - MLE the Exponential Distribution Rate Parameter
***

Suppose you're given $m$ numbers $x_1, x_2, \ldots, x_m$ (think training data) and told that they're samples from the exponential distribution $Exp(\lambda)$ where the rate parameter $\lambda$ is unknown. Recall that the probability density function for $Exp(\lambda)$ is given by

$$
f_\lambda(x) = \left\{
\begin{array}{rl}
0 & \textrm{if } x < 0 \\
\lambda e^{-\lambda x} & \textrm{if } x \geq 0
\end{array}
\right.
$$

In this problem we'll use Maximum Likelihood Estimation to estimate the rate parameter by hand and with a simulation

**Q1.1 [5pts]** Write down the likelihood function $L(\lambda)$ for the data set $x_1, x_2, \ldots, x_m$.


% BEGIN

% YOUR ANSWER HERE

% END

**Q1.2 [5pts]** Write down the associated Negative Log-Likelihood $\textrm{NLL}(\lambda)$ and simplify it algebraically.


% BEGIN

% YOUR ANSWER HERE

% END

**Q1.3 [5pts]** Find a formula for the MLE of the rate parameter $\lambda$ by taking the derivative of $\textrm{NLL}(\lambda)$, setting it equal to zero, and solving for $\hat{\lambda}$.


% BEGIN

% YOUR ANSWER HERE

% END

**Q1.4 [5pts]** Use the formula you found in **Q1.3** to estimate the rate parameter $\lambda$ for `x_train` data for each `m`. Plot and comment on the results.

In [ ]:
lam = 0.1
Ms = np.arange(1, 100) * 100

lam_hat = Ms * 0.0
for i, m in enumerate(Ms):
    x_train = np.random.exponential(1 / lam, size=m) + np.random.uniform(-.1, .1, m)
    # Workspace 1.4
    # BEGIN 
    # code here
    # END

% BEGIN

% YOUR ANSWER HERE

% END

### Datasets

For the remaining questions, we'll be using three synthetic datasets plotted below. From left to right: co-centric circles `circles`, blobs with 5 centers `multi_blobs`, and blobs with 2 centers `binary_blobs`.

Each of the dataset instances has (`X`, `labels`) attributes that are split into `X_train, y_train` and `X_test, y_test` partitions.

In [ ]:
%matplotlib inline
# Do not modify this cell
circles = data.Circles()
multi_blobs = data.DataBlobs(centers=4, std=1.5)
binary_blobs = data.DataBlobs(centers=2, std=1.5)

fig, axs = plt.subplots(1, 3)
fig.set_figheight(4), fig.set_figwidth(14)
for i, (dataset, name) in enumerate([(multi_blobs, "multi-blobs"),
                                     (circles, "Co-centric circles"),
                                     (binary_blobs, "binary blobs")]):
    axs[i].set_title(name)
    axs[i].scatter(dataset.X[:, 0], dataset.X[:, 1], c=dataset.labels)
plt.show()

# Problem 2 : Gaussian Mixture Model (40 points)

In a similar way to Problem 1, Gaussian Mixture Models (GMM) are based on the assumption that the data is generated from a certain distribution (mixture of $k$ Gaussian in this case), i.e:
There exists Gaussians $\{\mathcal{N}(\mu_1, \Sigma_1)\cdots \mathcal{N}(\mu_k, \Sigma_k)\}$ so that each sample $x_i$ is generated from one of the Gaussians. We work on $\mathbb{R}^2$

#### Notations:
- $\mu := (\mu_1,...\mu_k)$
- $\Sigma := (\Sigma_1,... \Sigma_k)$
- $p_i$ the probability density function of $\mathcal{N}(\mu_i, \Sigma_i)$
- $\pi=(\pi_1,...\pi_k)\in(0,1)^k$ where $\pi_i$ is the likelihood to generate samples from$\mathcal{N}(\mu_i, \Sigma_i)$ (we can interpret $\pi_i$ as the ratio of samples generated by the Gaussian $\mathcal{N}(\mu_i, \Sigma_i)$).
- $\alpha_k(x) := \frac{\pi_k. p_i(x)}{\sum_i \pi_i. p_i(x)}$: the probabilty that data point $x$ was generated by the k-th Gaussian (You can check that $\sum_i \alpha_i(x) = 1$)

Given data $\mathcal{D}=\{x_1,..., x_m\}$ the expected log-likelihood is:
\begin{align}
\ell (\pi,\mu,\Sigma) = \sum_{i=1}^{m}\sum_{j=1}^{k} \alpha_j(x_i)\left[\log \pi_j + \log p_j(x_i)\right]
\end{align}
(we use tha natural log)

**Q2.1[5pts]** Complete `sample_ll` to return the log likelihood of a single sample.

**Q2.3[5pts]** Complete `assign` to return the index of the Gaussian that is most likely to have generated the sample.
_Hint_: Use `GaussianMixture.alpha`


In [ ]:
from scipy.stats import multivariate_normal

class NormalDist:
    """Class Implementing a Gaussian"""
    def __init__(self, mu=None, Sigma=None):
        self.mu = mu or np.random.normal(0, 1, 2)
        self.sig = Sigma or np.diag([0.5,0.5])

    def pdf(self, x):
        """ return the pdf of N(mu, Sigma) applied to x
        Returns: array of shape (x.shape[0])
        """
        return multivariate_normal(mean=self.mu, cov=self.sig).pdf(x)


class GaussianMixture:
    """Class Implementing a Gaussian Mixture"""
    def __init__(self, k):
        """
        Args:
            k: Number of mixtures
        """
        self.k = k
        self.pi = np.random.uniform(0, 1, k)
        self.pi /= np.sum(self.pi)
        self.gaussians = [NormalDist() for _ in range(k)]

    def log_likelihood(self, X: np.ndarray):
        return sum([self.sample_ll(d) for d in X])

    def alpha(self, X):
        # we add small value to avoid overflow
        p = np.array([g.pdf(X) for g in self.gaussians]).T + 1e-8
        alpha =  p * self.pi[None, :]
        return  alpha / np.sum(alpha, axis=-1, keepdims=True)

    def sample_ll(self, x):
        # Workspace 2.1
        log_like = 0.0
        # BEGIN 
        # code here
        # END
        return log_like

    def assign(self, X):
        clusters = None
        # Workspace 2.2
        # BEGIN 
        # code here
        # END
        return clusters

In [ ]:
np.random.seed(4622)
gmm = GaussianMixture(4)
assert np.isclose(gmm.sample_ll(np.array([-.5,0.5])), -3.179732)


We use Expectation-Maximization to learn $\pi, (\mu_1, \Sigma_1),...(\mu_k, \Sigma_k)$. We assume that $\Sigma_i = diag(\sigma_{i, 1}, \sigma_{i,2})$ is diagonal.

Starting from randomly initialized parameters, we iterate the two steps:
- E step: For each sample $x$, compute $\alpha_k(x)$ (likelihood that $x$ is generated by the Gaussian k)
- M step: Update $(\pi, \mu, \Sigma)$ to maximize the likelihood:
\begin{align}
\sigma_{l,j}^2 &\leftarrow \frac{1}{\sum_i \alpha_l(x_i)} \sum_i \alpha_l(x_i) (x_{i,j} - \mu_{l,j})^2 \\
\mu_k &\leftarrow \frac{1}{\sum_i \alpha_k(x_i)} \sum_i \alpha_k(x_i) x_i\\
\pi_k(x_i) & \leftarrow \frac{\sum_i \alpha_k(x_i) }{m}
\end{align}
The iteration stops when the improvement ratio of the likelihood, `|new_likelihood - previous_likelihood/previous_likelihood |` is  less than `rtol`

**Q2.3[15pts]** Complete `?_update` methods of GMM to implement the M step.

**Q2.4[8pts]** Complete `fit` method by calling `?_update` methods and checking if the termination criterion is satisfied.

In [ ]:
class GMM(GaussianMixture):
    def __init__(self, k, rtol=1e-4):
        super().__init__(k)
        self.plots = []
        self.rtol = rtol
        self.snapshots = []

    def mu_update(self, alpha, X):
        # Workspace 2.3.a
        # BEGIN 
        # code here
        # END
    def sig_update(self, alpha, X):

        # Workspace 2.3.a
        # BEGIN 
        # code here
        # END

    def pi_update(self, alpha):
        # Workspace 2.3.a
        # BEGIN 
        # code here
        # END

    def fit(self, X):
        """
        Implement the GMM algorithm here as described above. Loop until the improvement ratio of the objective
        is lower than rtol. At the end of each iteration, save the GMM objective and return the objective values
        at the end

        @param X:
        @return:
        """
        objective = self.log_likelihood(X)
        history = [objective]
        # Workspace 1.5

        while True:
            self.save_snapshot(X, self.predict(X))
            alpha = self.alpha(X)
            # BEGIN 
            # code here
            # END
            history.append(self.log_likelihood(X))
        return history

    def predict(self, X):
        return self.assign(X)

    def save_snapshot(self, X, assignments):
        """
        Saves plot image of the current assignments
        """
        if X.shape[1] == 2:
            self.snapshots.append(helpers.create_buffer(X, assignments))

In [ ]:
# show progress code
%matplotlib notebook
np.random.seed(4622)
gmm = GMM(4)
objective_history = gmm.fit(multi_blobs.X)
helpers.show_progress(gmm.snapshots)

In [ ]:
%matplotlib inline
plt.plot(objective_history)
plt.show()


**Q2.5 [7 points]** For each of the three datasets (`circles`, `multi_blobs`, and `binary_blobs`), do the following on the `training` partition of the data:
- Fit a GMM with $k$ = # of unique labels.
- Produce a scatter plot of the data points and color the points according to their *predicted labels*.

Comment on the results

In [ ]:
%matplotlib inline
np.random.seed(42)
fig, axs = plt.subplots(1, 3)
fig.set_figheight(3), fig.set_figwidth(10)
for i,(dataset, name, k) in enumerate([(circles, "Cocentric circles", 2),
                                    (multi_blobs, "multi-blobs", 4),
                                    (binary_blobs, "binary blobs", 2)]):
    # Workspace 2.5
    # BEGIN 
    # code here
    # END

plt.show()

% BEGIN

% YOUR ANSWER HERE

% END

## Evaluating GMM (5 Bonus pts)
The easiest way to evaluate the clustering quality is to use the true labels. The natural question here is: which cluster corresponds to which label?

Let's first formulate the question using `multi_blobs` dataset. We have 4 clusters and 4 classes in our data. Let's create a _confusion matrix_ $C$ between the clusters and the labels so that $ C_{i,j} = \text{size}(\text{cluster}_i \cap \text{class}_j)$.

We model the unknown mapping using the $4\times 4$ boolean matrix $X$ such that $X_{i,j}=1$ if and only if $\text{cluster}_i$ is mapped to $\text{class}_j$.

To avoid having a cluster containing multiple classes, each row of $X$ is constrained to have only one non-zero entry.

Now, given a mapping $X$ and confusion matrix $C$, the number of correctly "classified" points (not really classification, more of clustering here) is:

\begin{align}
\#\text{correct} = \sum_i \sum_j C_{i,j} X_{i,j}
\end{align}
The goal is to find $\hat{X}$ that maximizes $\#\text{correct}$. To solve for $X$ we're going to use `scipy`'s `linear_sum_assignment`
([documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html)).

- **Q2.6 [4 points]** Complete `evaluate_clustering` to return the accuracy of GMM(4) using the optimal mapping $\hat{X}$ defined above.

In [ ]:
from scipy.optimize import linear_sum_assignment
def evaluate_clustering(trained_model, X, labels):
    clusters = trained_model.predict(X)
    # Workspace 2.6
    # BEGIN 
    # code here
    # END
    return accuracy

**Q2.7 [1 points]** Run GMM(4) on the full `multi_blobs` for 5 times. report the mean clustering performance and the standard deviation

In [ ]:
%matplotlib inline
accuracies = []
# Workspace 2.7

for _ in range(5):
    gmm = GMM(4)
    # BEGIN 
    # code here
    # END

# Problem 3: Classification using Support Vector Machines and Kernel Trick (40 points)

We have seen during the class the dual form of the Support Vector Machine problem using a kernel $K$:

\begin{aligned}
 \max_{\alpha} \sum_i^m \alpha_i &- \frac{1}{2} \sum_{i,j}^m y^{(i)}y^{(j)} \alpha_i \alpha_j K(x^{(i)},x^{(j)})
    \\
      s.t. \text{   } \alpha_i &\geq 0 \\
      \sum_i^m \alpha_i y^{(i)} &= 0
\end{aligned}

The simplest kernel $K$ is the linear kernel 
\begin{align}
K_{lin}(x^{(i)},x^{(j)}) = <x^{(i)}, x^{(j)}>
\end{align}
 with $<.,.>$ being the scalar product.

 We'll be also using the radial kernel $K_{rad}$:
\begin{align}
 K_{rad, \gamma}(x^{(i)},x^{(j)})  = \exp \big[-\gamma ||x^{(i)} - x^{(j)}||^2]
\end{align}
And the polynomial kernel $K_{poly, c, p}$:
\begin{align}
 K_{poly, c, p}(x^{(i)},x^{(j)})  = (<x^{(i)}, x^{(j)}> + c)^p
\end{align}

**Q3.1. [12 pts]** Complete the implementation of the three kernels defined above

In [ ]:
class LinearKernel(object):
    def compute(self, x1, x2):
        """
        Compute the kernel matrix
        @param x1: array of shape (m1,p)
        @param x2: array of shape(m2,p)
        @return: K of shape (m1,m2) where K[i,j] = <x1[i], x2[j]>
        """
        # Workspace 3.1.a
        K = np.zeros((x1.shape[0], x2.shape[0]))
        # BEGIN 
        # code here
        # END
        return K


class RadialKernel(object):

    def __init__(self, gamma):
        self.gamma = gamma

    def compute(self, x1, x2):
        """
        Compute the kernel matrix. Hint: use sklearn's Euclidean distance 
        @param x1: array of shape (m1,p)
        @param x2: array of shape(m2,p)
        @return: K of shape (m1,m2) where K[i,j] = K_rad(x1[i],x2[j]) = exp(-gamma * ||x1[i] - x2[j]||^2)
        """
        # Workspace 3.1.b
        K = np.zeros((x1.shape[0], x2.shape[0]))
        # BEGIN 
        # code here
        # END
        return K


class PolynomialKernel(object):

    def __init__(self, c, p):
        self.c = c
        self.p = p

    def compute(self, x1, x2):
        """
        Compute the kernel matrix.
        @param x1: array of shape (m1,p)
        @param x2: array of shape(m2,p)
        @return: K of shape (m1,m2) where K[i,j] = (x1[i].x2[j] + c)^p
        """
        # Workspace 3.1.c
        K = np.zeros((x1.shape[0], x2.shape[0]))
        # BEGIN 
        # code here
        # END
        return K

Now we'll solve for $\alpha$ of the dual form  using the quadratic solver from [`cvxopt` package](https://cvxopt.org/userguide/coneprog.html#quadratic-programming).
To match the solver API, we need to rewrite the problem as:

\begin{aligned}
    \min \frac{1}{2} x^TPx + q^Tx
    \\
     s.t. \ Gx \leq h
    \\
    \ Ax = b
\end{aligned}

So we'll define $P, q, G, h, A, b$  as:

\begin{align}
P_{i,j} &= y^{(i)}y^{(j)} K(x^{(i)},x^{(j)}) \text{,  matrix $P$ is of shape $m\times m$}\\
q &= -\overline{1} \text{,  vector of size m} \\
G &= diag(-\overline{1}) = -I_m\text{, diagonal matrix of -1} \\
h &= \overline{0} \text{,  vector of size m} \\
A &= y \text{, the labels vector} \\
b &= 0 \text{, a scalar (not to confuse with the intercept of SVM)}
\end{align}

- **3.2 [8 points]** Complete `quadratic_solver` by defining the different arrays $P, q, G, h, A$ and the scalar $b$. Make sure all arrays are of float type (it is important for `cvxopt`).


In [ ]:
from cvxopt import matrix
from cvxopt import solvers
import numpy as np

solvers.options['show_progress'] = False
solvers.options['abstol'] = 1e-10
solvers.options['reltol'] = 1e-10
solvers.options['feastol'] = 1e-10


def quadratic_solver(K, y, beta=0.0):
    """
    Solve for alpha of the dual problem, 
    @param K: Kernel matrix K of shape (m,m)
    @param y: labels array y of shape (m,)
    @return: optimal alphas of shape (m,)
    """

    # Workspace 3.2
    m = K.shape[0]
    #P = ? # shape (m,m)
    #q = ? # shape(m,1)
    #G = ? # shape(m,m)
    #h = ? # shape (m,)
    #A = ? # shape (1,m)
    #b = ? # scalar
    # BEGIN 
    # code here
    # END
    h = np.zeros(m)  #shape (m,)
    A = y.reshape(1, -1)  # shape (1,m)
    b = 0.0  # scalar
    sol = solvers.qp(P=matrix(P.astype(float)),
                     q=matrix(q.astype(float)),
                     G=matrix(G.astype(float)),
                     h=matrix(h.astype(float)),
                     A=matrix(A.astype(float)),
                     b=matrix(b))
    alphas = np.array(sol['x'])
    alphas = alphas * (np.abs(alphas) > 1e-8)  # zeroing out the small values
    return alphas.reshape(-1)

Once we get the optimal $\alpha$, then we can get the indices of the support vectors $S = \{i | \alpha_i >0 \}$. The intercept $b$ is computed as:

\begin{align}
b = \frac{1}{|S|}\sum_{m\in S}\big[ y^{(m)} - \sum_{i\in S} \alpha_i  y^{(i)}K(x^{(i)}, x^{(m)})\big],
\end{align}
(we only sum over the support vectors, i.e for which $\alpha>0$)

and the prediction for a point $x$ would be:

\begin{align}
\hat{y} = \text{sign}\big[\sum_i y^{(i)}\alpha_i K(x,x^{(i)}) + b \big],
\end{align}
The sum is originally over the support vectors, since for non-support vectors $\alpha=0$ we can sum over the entire training data.

You can see that we only need access to the kernel $K$ and the model is oblivious to the features themselves. That's the power of kernel methods!

- **3.3 [8 points]** Complete the `fit` method of SVM. You have to transform the 0-1 $y$ labels to (-1,1) before doing the computations since the SVM formulation assumes that the binary labels are (-1,1).

- **3.4 [5 points]** Complete the `predict` method to return the predicted labels of the provided points.


In [ ]:
class SVM(object):

    def __init__(self, kernel, beta=0.0):
        self.kernel = kernel
        self.X = None
        self.y = None
        self.intercept = None
        self.alphas = None
        self.beta = beta  # ridge regularization coefficient

    def fit(self, X, y):
        """
        Transform y to (-1,1) and use self.kernel to compute K
        Solve for alphas and compute the intercept using the provided expression
        Keep track of X and y since you'll need them for the prediction
        @param X: data points of shape (m,p)
        @param y: (0,1) labels of shape (m,)
        @return: None
        """
        # Workspace 3.3
        self.X = X
        self.y = 2 * y - 1
        # BEGIN 
        # code here
        # END

    def predict(self, X):
        """
        Predict the labels of points in X
        @param X: data points of shape (m,p)
        @return: predicted 0-1 labels of shape (m,)
        """
        # Workspace 3.4
        predicted_labels = np.zeros((X.shape[0],))
        # BEGIN 
        # code here
        # END
        return predicted_labels

We provide below an example of the expected plots for this problem using the linear kernel. 

- **3.5 [2 points]** Edit the cell to report the accuracy on the test sets for each of the two datasets visualized in the plots. How do you explain the obtained accuracies?

In [ ]:
# BEGIN 
# code here
# END

% BEGIN

% YOUR ANSWER HERE

% END

- **3.6 [3 points]** Plot and report SVM performance on the same datasets as in 2.5 using the radial kernel with $\gamma=2.0$. Describe the model performance and compare it to the linear kernel.

In [ ]:
# BEGIN 
# code here
# END

% BEGIN

% YOUR ANSWER HERE

% END

- **3.7 [2 points]** Plot and report SVM performance on the same datasets as in 2.5 using the polynomial kernel with $(c,p) = (1,5)$. Describe the model performance and compare it to the two previous kernels

In [ ]:
# BEGIN 
# code here
# END

% BEGIN

% YOUR ANSWER HERE

% END

### Regularization (4 Bonus pts)

We add a ridge regularization of $\alpha$ with coefficient $\beta$, this time there is a minus $-$ since we're maximizing (so that $\beta ||\alpha||^2$ is minimized).
\begin{aligned}
 \max_{\alpha} \Big[ \sum_i^m \alpha_i &- \frac{1}{2} \sum_{i,j}^m y^{(i)}y^{(j)} \alpha_i \alpha_j K(x^{(i)},x^{(j)}) - \frac{\beta}{2} ||\alpha||^2 \Big]
    \\
      s.t. \text{   } \alpha_i &\geq 0 \\
      \sum_i^m \alpha_i y^{(i)} &= 0
\end{aligned}

**Q3.8 [Bonus, 4pts]** Which elements of the quadratic system $P, q, G, h, A, b$ do we have to Change? Edit **Q3.2** and `SVM` to incorporate `beta` argument.